<a href="https://colab.research.google.com/github/Chirag314/Interpretable-machine-learning/blob/main/Flight_Delays.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install interpret
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())

In [ ]:
!pip install gaminet

In [ ]:
!pip install rulefit

In [ ]:
import math

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures,StandardScaler,MinMaxScaler
from sklearn import metrics,linear_model,tree,naive_bayes,neighbors,ensemble,neural_network,svm
from rulefit import RuleFit
import statsmodels.api as sm
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show
from interpret.perf import ROC
import tensorflow as tf
from gaminet import GAMINet
from gaminet.utils import plot_trajectory, local_visualize, global_visualize_density, plot_regularization,feature_importance_visualize



In [ ]:
!pip install --upgrade machine-learning-datasets
import machine_learning_datasets as mldatasets

In [ ]:
aad18_df=mldatasets.load("aa-domestic-delays-2018")

https://github.com/PacktPublishing/Interpretable-Machine-Learning-with-Python/raw/master/datasets/aa-domestic-delays-2018.csv.zip downloaded to /content/data/aa-domestic-delays-2018.csv.zip
/content/data/aa-domestic-delays-2018.csv.zip uncompressed to /content/data/aa-domestic-delays-2018.csv
1 dataset files found in /content/data/aa-domestic-delays-2018.csv folder
parsing /content/data/aa-domestic-delays-2018.csv/aa-domestic-delays-2018.csv


In [ ]:
print(aad18_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899527 entries, 0 to 899526
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   FL_NUM                899527 non-null  int64  
 1   ORIGIN                899527 non-null  object 
 2   DEST                  899527 non-null  object 
 3   PLANNED_DEP_DATETIME  899527 non-null  object 
 4   CRS_DEP_TIME          899527 non-null  int64  
 5   DEP_TIME              899527 non-null  float64
 6   DEP_DELAY             899527 non-null  float64
 7   DEP_AFPH              899527 non-null  float64
 8   DEP_RFPH              899527 non-null  float64
 9   TAXI_OUT              899527 non-null  float64
 10  WHEELS_OFF            899527 non-null  float64
 11  CRS_ELAPSED_TIME      899527 non-null  float64
 12  PCT_ELAPSED_TIME      899527 non-null  float64
 13  DISTANCE              899527 non-null  float64
 14  CRS_ARR_TIME          899527 non-null  int64  
 15  

FL_NUM: Flight number.
ORIGIN: Starting airport code (IATA).
DEST: Destination airport code (IATA).
Departure features are as follows:

PLANNED_DEP_DATETIME: The planned date and time of the flight.
CRS_DEP_TIME: The planned departure time.
DEP_TIME: The actual departure time.
DEP_AFPH: The number of actual flights per hour occurring during the interval in between the planned and actual departure from the origin airport (factoring in 30 minutes of padding). The feature tells you how busy the origin airport was during takeoff.
DEP_RFPH: The departure relative flights per hour is the ratio of actual flights per hour over the median number of flights per hour that occur at the origin airport at that time of day, day of the week, and month of the year. The feature tells you how relatively busy the origin airport was during takeoff.
TAXI_OUT: The time duration elapsed between the departure from the origin airport gate and wheels off.
WHEELS_OFF: The point in time that the aircraft’s wheels leave the ground.
In-flight features are as follows:

CRS_ELAPSED_TIME: The planned amount of time needed for the flight trip.
PCT_ELAPSED_TIME: The ratio of actual flight time over planned flight time to gauge the plane’s relative speed.
DISTANCE: The distance between two airports.
Arrival features are as follows:

CRS_ARR_TIME: The planned arrival time.
ARR_AFPH: The number of actual flights per hour occurring during the interval between the planned and actual arrival time at the destination airport (factoring in 30 minutes of padding). The feature tells you how busy the destination airport was during landing.
ARR_RFPH: The arrival relative flights per hour is the ratio of actual flights per hour over the median number of flights per hour that occur at the destination airport at that time of day, day of the week, and month of the year. The feature tells you how relatively busy the destination airport was during landing.
Delay features are as follows:

DEP_DELAY: The total delay on departure in minutes.
ARR_DELAY: The total delay on arrival in minutes can be subdivided into any or all of the following:
CARRIER_DELAY: The delay in minutes caused by circumstances within the airline’s control (for example, maintenance or crew problems, aircraft cleaning, baggage loading, fueling, and so on).
WEATHER_DELAY: The delay in minutes caused by significant meteorological conditions (actual or forecasted).
NAS_DELAY: The delay in minutes mandated by a national aviation system such as non-extreme weather conditions, airport operations, heavy traffic volume, and air traffic control.
SECURITY_DELAY: The delay in minutes caused by the evacuation of a terminal or concourse, re-boarding of an aircraft because of a security breach, faulty screening equipment, or long lines above 29 minutes in screening areas.
LATE_AIRCRAFT_DELAY: The delay in minutes caused by a previous flight with the same aircraft that arrived late.

In [ ]:
aad18_df['PLANNED_DEP_DATETIME']= pd.to_datetime(aad18_df['PLANNED_DEP_DATETIME'])

In [ ]:
#create features for month and day
aad18_df['DEP_MONTH']=aad18_df['PLANNED_DEP_DATETIME'].dt.month
aad18_df['DEP_DOW']=aad18_df['PLANNED_DEP_DATETIME'].dt.dayofweek

In [ ]:
aad18_df=aad18_df.drop(['PLANNED_DEP_DATETIME'],axis=1)

In [ ]:
#Create list with 10 hubs
hubs=['CLT' ,'ORD','DFW','LAX','MIA','JFK','LGA','PHL','PHX','DCA']
is_origin_hub=aad18_df['ORIGIN'].isin(hubs)
is_dest_hub=aad18_df['DEST'].isin(hubs)
aad18_df['ORIGIN_HUB']=0
aad18_df.loc[is_origin_hub,'ORIGIN_HUB']=1
aad18_df['DEST_HUB']=0
aad18_df.loc[is_dest_hub,'DEST_HUB']=1
aad18_df=aad18_df.drop(['FL_NUM','ORIGIN','DEST'],axis=1)

After all these operations, we have a fair number of useful features, but we are yet to determine the target feature. There are two columns that could serve this purpose. We have ARR_DELAY, which is the total number of minutes delayed regardless of the reason, and then there’s CARRIER_DELAY, which is just the total number of those minutes that can be attributed to the airline. For instance, look at the following sample of flights delayed over 15 minutes (which is considered late according to the airline’s definition)

In [ ]:
aad18_df.loc[aad18_df['ARR_DELAY']>15,['ARR_DELAY','CARRIER_DELAY']].head(10)

,ARR_DELAY,CARRIER_DELAY
8,168.0,136.0
16,20.0,5.0
18,242.0,242.0
19,62.0,62.0
22,19.0,19.0
26,26.0,0.0
29,77.0,77.0
32,19.0,19.0
33,18.0,1.0
40,36.0,16.0


one of them (#26) wasn’t at all the responsibility of the airline because only 0 minutes could be attributed to the airline. Four of them were partially the responsibility of the airline (#8, #16, #33, and #40), two of which were over 15 minutes late due to the airline (#8 and #40). The rest of them were entirely the airline’s fault. We can tell that although the total delay is useful information, the airline executives were only interested in delays caused by the airline so ARR_DELAY can be discarded.

In [ ]:
aad18_df=aad18_df.drop(['ARR_DELAY'],axis=1)

In [ ]:
np.random.seed(42)
y=aad18_df['CARRIER_DELAY']
X=aad18_df.drop(['CARRIER_DELAY'],axis=1).copy()
X_train,X_test,y_train_reg,y_test_reg=train_test_split(X,y,test_size=0.15,random_state=42)
y_train_class=y_train_reg.apply(lambda x:1 if x>15 else 0)
y_test_class=y_test_reg.apply(lambda x :1 if x>15 else 0)

To examine how linearly correlated the features are to the target CARRIER_DELAY, we can compute Pearson’s correlation coefficient, turn coefficients to absolute values (because we aren’t interested in whether they are positively or negatively correlated), and sort them in descending order:

In [ ]:
corr=aad18_df.corr()
abs(corr['CARRIER_DELAY']).sort_values(ascending=False)

CARRIER_DELAY          1.000000
DEP_DELAY              0.703935
ARR_RFPH               0.101742
LATE_AIRCRAFT_DELAY    0.083166
DEP_RFPH               0.058659
ARR_AFPH               0.035135
DEP_TIME               0.030941
NAS_DELAY              0.026792
WHEELS_OFF             0.026787
TAXI_OUT               0.024635
PCT_ELAPSED_TIME       0.020980
CRS_DEP_TIME           0.016032
ORIGIN_HUB             0.015334
DEST_HUB               0.013932
DISTANCE               0.010680
DEP_MONTH              0.009728
CRS_ELAPSED_TIME       0.008801
DEP_DOW                0.007043
CRS_ARR_TIME           0.007029
DEP_AFPH               0.006053
WEATHER_DELAY          0.003002
SECURITY_DELAY         0.000460
Name: CARRIER_DELAY, dtype: float64

 this is only linearly correlated and on a one-by-one basis. It doesn’t mean that they don’t have a non-linear relationship

Check with various regression models


In [ ]:
reg_models={
    'linear':{'model':linear_model.LinearRegression()},
    'linear_poly':{
        'model':make_pipeline(
            PolynomialFeatures(degree=2),
            linear_model.LinearRegression(fit_intercept=False)
        )
    },
    'linear_interact':{
        'model':make_pipeline(
            PolynomialFeatures(interaction_only=True),
            linear_model.LinearRegression(fit_intercept=False)
        )
    },
    'ridge':{
        'model':linear_model.RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])
    },
    'decision_tree':{
        'model':tree.DecisionTreeRegressor(
            max_depth=7,random_state=42
        )
    },
    'rulefit':{
        'model': RuleFit(
             max_rules=150,
             rfmode='regress',
             random_state=42
        )
    },
    'knn':{
        'model':neighbors.KNeighborsRegressor(n_neighbors=7)
    },
    'random_forest':{
        'model':ensemble.RandomForestRegressor(
            max_depth=7,random_state=42
        )
    },
    'mld':{
        'model':neural_network.MLPRegressor(
            hidden_layer_sizes=(21,),
            max_iter=500,
            early_stopping=True,
            random_state=42

        )
    }
}

In [ ]:
%%time
for model_name in reg_models.keys():
    print(model_name)
    if model_name != 'rulefit':
        fitted_model = reg_models[model_name]['model'].fit(X_train, y_train_reg)
        y_train_pred = fitted_model.predict(X_train)
        y_test_pred = fitted_model.predict(X_test)
    else:
        fitted_model = reg_models[model_name]['model'].fit(X_train.values, y_train_reg.values, X_test.columns)
        y_train_pred = fitted_model.predict(X_train.values)
        y_test_pred = fitted_model.predict(X_test.values)
    reg_models[model_name]['fitted'] = fitted_model
    reg_models[model_name]['preds'] = y_test_pred
    reg_models[model_name]['RMSE_train'] = math.sqrt(metrics.mean_squared_error(y_train_reg, y_train_pred))
    reg_models[model_name]['RMSE_test'] = math.sqrt(metrics.mean_squared_error(y_test_reg, y_test_pred))
    reg_models[model_name]['R2_test'] = metrics.r2_score(y_test_reg, y_test_pred)


linear
linear_poly
linear_interact
ridge
decision_tree
rulefit
knn
random_forest
mld
CPU times: user 2h 19min 49s, sys: 1min 39s, total: 2h 21min 29s
Wall time: 1h 28min 45s
